# Import dependencies

In [ ]:
# To mute annoying warnings in notebook
import warnings

import numpy as np
import pandas as pd
import keras
from keras import layers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

# Getting data, observations
Get dataset

In [ ]:
# Get dataset from file
df = pd.read_csv(
    f"../data/IMDB_Dataset.csv",
)

# Show dataset head
df.head()

In [ ]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200 

In [ ]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
# Set feature and target subsets
features = df.review
target = df.sentiment

In [ ]:
tf_idf = TfidfVectorizer(stop_words='english')
features_vector = tf_idf.fit_transform(features)

pd.DataFrame.sparse.from_spmatrix(
    data=features_vector,
    index=features.index,
    columns=tf_idf.get_feature_names_out()
)

In [ ]:
# # Get split subsets
# X_train, X_test, y_train, y_test = train_test_split(
#     features_vector,
#     target_vector,
#     test_size=0.3,
#     random_state=42
# )

In [ ]:
# (x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
#     num_words=max_features
# )
# print(len(x_train), "Training sequences")
# print(len(x_val), "Validation sequences")
# # Use pad_sequence to standardize sequence length:
# # this will truncate sequences longer than 200 words and zero-pad sequences shorter than 200 words.
# x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
# x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)